# Manual Relation/Alignment Prototype in Jupyter NB
Purpose: Explore options for inputing relations

In [4]:
# %pip install ipympl
# %pip install pyqt5

In [9]:
import sys
import os
sys.path.insert(0, os.path.abspath("../.."))  # Repository directory relative to this file.
from MISalign.model.project import Project
from MISalign.model.relation import Relation
from MISalign.model.relation_service import relation_tree
from MISalign.model.image import Image

from PIL import Image as PILImage

from matplotlib import pyplot as plt

In [11]:
from os.path import abspath

In [17]:
%matplotlib qt5
img=PILImage.open(abspath(r"..\..\example\data\set_a\a_myimages01.jpg"))
# display(img)
plt.imshow(img)
plt.ginput(5)
# select up to 5 points using left mouse
# stop collecting points with middle mouse
# undo previous with right click

[(773.5259740259742, 611.1883116883117),
 (1043.0064935064936, 624.1753246753248),
 (851.4480519480521, 789.7597402597403),
 (1039.7597402597405, 783.2662337662338),
 (1231.318181818182, 491.0584415584416)]

[(952.0974025974028, 604.6948051948052),
 (744.3051948051949, 815.7337662337663)]